<a href="https://colab.research.google.com/github/ormorteey/STA-237-Time-Series/blob/main/EDA_%26_Scalogram_STA_237_Project_%5BTime_Series_Anomaly_Detection_Using_Computer_Vision%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
import skimage.measure
import pywt
import os
import time
from google.colab import files

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.112.175.178:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.112.175.178:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%capture
%%R
# Signify cell is an R cell
# Silence Output

h = install.packages("pacman")
h = library(pacman)

(as ‘lib’ is unspecified)







	‘/tmp/RtmpPNLJBr/downloaded_packages’



In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
%%R
# load packages for data manipulation in R
p_load("tidyverse", "googledrive")

In [ ]:
# Silence Output
%%capture
%%R


# killing auth request
drive_deauth()
drive_user()

# retrieving fulll url ID of files
public_file = drive_get(as_id(c("10i8tM37aqHvD-YH2JK1qZAXzYdawIsgh","1eJ4uXiFqGqsXJ2ut5vex_Lg3BLlUAPjT","1B6pbF90ryhJ-kpu0nQ3FKt6RcSavgCMS")) )

# download the files
save_output = 1:3 %>% map(~drive_download(public_file[.,], overwrite = T))

# file location for HDF5 & csv datasets
airbus_train_path = 'dftrain.h5'
airbus_valid_path = 'dfvalid.h5'
airbus_valid_groundtruth_path = 'dfvalid_groundtruth.csv'

  

In [ ]:
def prepare_scalograms(df, rows, cols):
  
  # cast df as numpy array
  df_tensor = df.to_numpy()
  # reshape df to tensor to rows of df with 120 series with length 512
  df_tensor = np.reshape(df_tensor, (df.shape[0],120, 512))
  # diagnostics: shape of dataframe
  print(df.shape)
  # diagnostics: shape of df tensor
  print(df_tensor.shape)
  # get continuous wavelet transform scales
  SC_scales = np.array([2**(i/4) for i in range(1,65)])

  print(SC_scales.shape)

  # create empty scalograms
  df_scalograms = np.empty([rows, cols, 64, 64])
  # fill up scalogram
  for ii in np.arange(rows):
    start_time = time.time()
    for jj in np.arange(cols):     

      df_scalograms[ii,jj,:,:] =  get_scalogram(df_tensor[ii,jj], SC_scales)
      end_time = time.time()
      if (jj % (cols/2)) == 0:
          print("row {} / column {}. Time elaspse from column 0 {} secs".format(ii, jj, np.round(end_time-start_time,2)))
  reshape_list = df_scalograms.shape
  print(df_scalograms.shape)
  df_scalograms = df_scalograms.reshape(reshape_list[0] * reshape_list[1], reshape_list[2], reshape_list[3], 1)

  return(df_scalograms)


In [ ]:
def get_scalogram(x, SC_scales):
  cwtmatr, freqs = pywt.cwt(x, SC_scales, 'mexh')
  a = skimage.measure.block_reduce(cwtmatr, (1,8), np.average)  
  return(a)

def scalogram_plotter(x, show = False):
  plt.imshow(x.reshape(64, 64), cmap = "jet", aspect = "auto")
  if show == True:
      plt.show()


In [ ]:

def prepare_dataset(rows_t = 1677, cols_t = 120, rows_v = 594, cols_v = 120):
  # read hdf files: training data and validation data and ground truth [labels]
  train_df = pd.read_hdf(airbus_train_path)
  valid_df = pd.read_hdf(airbus_valid_path)
  valid_groundtruth_df = pd.read_csv(airbus_valid_groundtruth_path)

  validation_examples = prepare_scalograms(valid_df, rows = rows_v, cols = cols_v)
  print("Completed validation examples")

  train_examples = prepare_scalograms(train_df, rows = rows_t, cols = cols_t)
  print("Completed training examples")

  np.savez_compressed('data_set', validation_examples, train_examples)

  validation_labels = valid_groundtruth_df['anomaly'].head(rows_v).to_numpy()

  train_labels = np.ones([train_examples.shape[0]])

  # create dataset to serve the model
  # train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
  # validation_dataset = tf.data.Dataset.from_tensor_slices((validation_examples, validation_labels))

  # Configure dataset batch size and shuffler
  # BATCH_SIZE = 10000
  # SHUFFLE_BUFFER_SIZE = 10000

  # train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  # train_dataset = train_dataset.batch(BATCH_SIZE)
  # validation_dataset = validation_dataset.batch(BATCH_SIZE)

  # Configuring dataset performance
  # AUTOTUNE = tf.data.AUTOTUNE

  # train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
  # validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)
  return(train_examples, train_labels, validation_examples, validation_labels)

In [ ]:
def get_model(tpu = False):

  
  model = tf.keras.Sequential([
                         
    tf.keras.layers.InputLayer(input_shape = (64,64,1)),                          
    tf.keras.layers.Conv2D(64, kernel_size=2 , strides=2, activation=tf.keras.layers.LeakyReLU(alpha=0.3)),
    tf.keras.layers.Conv2D(128, kernel_size=(2,2), strides=(2,2), activation=tf.keras.layers.LeakyReLU(alpha=0.3)),
    tf.keras.layers.Dense(32768,  activation=tf.keras.layers.LeakyReLU(alpha=0.3)),
    tf.keras.layers.Dense(300, activation=tf.keras.layers.LeakyReLU(alpha=0.3)),
    tf.keras.layers.Dense(32768, activation=tf.keras.layers.LeakyReLU(alpha=0.3)),
    tf.keras.layers.Conv2DTranspose(128, kernel_size=1, strides=1, activation=tf.keras.layers.LeakyReLU(alpha=0.3)),
    tf.keras.layers.Conv2DTranspose(64, kernel_size=2 , strides=2, activation=tf.keras.layers.LeakyReLU(alpha=0.3)),
    tf.keras.layers.Conv2DTranspose(1, kernel_size=2 , strides=2, activation=tf.keras.layers.LeakyReLU(alpha=0.3))

  ])


  #compile the model 
  model.compile(loss=tf.losses.MeanAbsoluteError(),
               optimizer='adam',
               metrics=['accuracy'])

  return model




In [ ]:
airbus_train_path = 'dftrain.h5'
airbus_valid_path = 'dfvalid.h5'
airbus_valid_groundtruth_path = 'dfvalid_groundtruth.csv'

with strategy.scope():
  rows_t, cols_t, rows_v, cols_v = 1677, 120, 594, 120
  # rows_t, cols_t, rows_v, cols_v = 4, 4, 4, 4
  # train_ds = prepare_dataset(rows_t = 1677, cols_t = 120, rows_v = 594, cols_v = 120)
  train_examples, train_labels, validation_examples, validation_labels = prepare_dataset(rows_t, cols_t, rows_v, cols_v)

(594, 61440)
(594, 120, 512)
(64,)
row 0 / column 0. Time elaspse from column 0 0.78 secs
row 0 / column 60. Time elaspse from column 0 47.76 secs
row 1 / column 0. Time elaspse from column 0 0.73 secs
row 1 / column 60. Time elaspse from column 0 44.9 secs
row 2 / column 0. Time elaspse from column 0 0.74 secs
row 2 / column 60. Time elaspse from column 0 45.0 secs
row 3 / column 0. Time elaspse from column 0 0.73 secs
row 3 / column 60. Time elaspse from column 0 44.98 secs
row 4 / column 0. Time elaspse from column 0 0.73 secs
row 4 / column 60. Time elaspse from column 0 44.77 secs
row 5 / column 0. Time elaspse from column 0 0.74 secs
row 5 / column 60. Time elaspse from column 0 44.75 secs
row 6 / column 0. Time elaspse from column 0 0.72 secs
row 6 / column 60. Time elaspse from column 0 44.9 secs
row 7 / column 0. Time elaspse from column 0 0.73 secs
row 7 / column 60. Time elaspse from column 0 44.81 secs
row 8 / column 0. Time elaspse from column 0 0.74 secs
row 8 / column 60

In [ ]:
with strategy.scope():
  model = get_model(tpu = True)
  model.fit(
    train_examples, train_labels,
    epochs=10
  )


In [ ]:
model.predict(train_examples)

In [ ]:

def resid_fn(pre_img, post_img):
  return(np.sum(np.absolute(pre_img - post_img)))
  

In [ ]:
def classificator(validation_examples, train_examples, rows_t, cols_t, rows_v, cols_v, qtl = 0.99, model = None):
  post_validation_exam = model.predict(validation_examples)
  # post = train_examples = model.predict(train_examples)
  cutoff = get_train_residuals_qtl(train_examples, rows_t, cols_t, qtl, model)
  res = []
  temp_res = []
  for ii in range(0,rows_v * cols_v, cols_v):
    print("===== Done {} out of {}".format(ii, rows_v) )
    for jj in range(cols_v):
      temp_res.append(resid_fn(validation_examples[ii+jj,:, :,:], post_validation_exam[ii+jj,:,:,:]))
    
    if max(temp_res) > cutoff:
      res.append(1)
    else:
      res.append(0) 

  return(res) 



In [ ]:
model.predict(validation_examples)

In [ ]:
def get_train_residuals_qtl(train_examples, rows, cols, qtl = 0.99, model = None):
  
  results = []
  res = model.predict(train_examples)
  for ii in range(rows*cols):
    results.append( resid_fn(res[ii,:,:,:], train_examples[ii,:,:,:]))
  plt.plot(results)
  results.sort()
  plt.plot(results)  
  return(results[int(np.floor(len(results) * qtl))])
  # return(results)


In [ ]:
pred_train = get_train_residuals_qtl(train_examples, rows_t, cols_t, qtl = 0.99, model = model)

In [ ]:

with strategy.scope():
  pred_labels = classificator(validation_examples, train_examples, rows_t, cols_t, rows_v, cols_v, qtl = 0.99, model = model)

In [ ]:
def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(validation_labels, pred_labels)))
  print("Precision = {}".format(precision_score(labels, predictions)))
  print("Recall = {}".format(recall_score(labels, predictions)))

In [ ]:
print_stats(pred_labels, validation_labels)

In [ ]:
files.download("data_set.npz")